This code has been executed three different times in order to obtain the corresponding ROI of each brain region (hippocampus, amygdala and thalamus). 

In [1]:
import os 
import numpy as np
import nibabel as nib
from radiomics import featureextractor, getFeatureClasses
import csv

In [2]:
nom_carpetes = [3077, 3078, 3079, 3121, 3329, 3536, 3542, 3543, 3544, 3076, 3148, 3157, 3164, 3171, 3265, 3288, 
                3289, 3457, 4972, 4979, 4980, 4996, 4998, 5007, 5009, 5013, 5014, 5018, 5021, 5042, 5056, 5057, 
                5063, 5064, 5065, 5066, 5086, 5087, 5097, 5102, 5104, 5113, 5114, 4970, 4971, 4978, 4985, 4986, 
                4987, 4993, 4994, 4997, 5008, 5019, 5020, 5032, 5033, 5034, 5041, 5058, 5098, 5103, 5105, 5378]

path_ = "/Users/elisabetsorribes/Desktop/TFG/ALL DATASETS/"

nom_temps= ['t1', 't2', 't3', 't4', 't5']

In [3]:
for i in range (len(nom_carpetes)):
    case = nom_carpetes[i]
    
    for temps in nom_temps:
        path_1 = str(case) + "/" + str(temps) + "/" + "regions.nii"
        data_path = os.path.join(path_,path_1)
        #print(data_path)
    
        if os.path.isfile(data_path):
            labels = nib.load(data_path) #Reads the label file where there is the brain parcellation of the case
            #print(labels.shape) #--> (256, 256, 36)
            hipp_labels = np.zeros(labels.shape, dtype=np.float64)
            #print(labels.get_data_dtype()) --> float64
            #print(labels.affine.shape) --> (4,4)
        
        
            #If hippocampus computed:
            #Right hippocampus 
            hipp_labels[np.where(labels.get_fdata()==23)] = 1
            hipp_labels[np.where(labels.get_fdata()==24)] = 1
            hipp_labels[np.where(labels.get_fdata()==25)] = 1
            hipp_labels[np.where(labels.get_fdata()==26)] = 1
            hipp_labels[np.where(labels.get_fdata()==27)] = 1

            #Left hippocampus
            hipp_labels[np.where(labels.get_fdata()==71)] = 1
            hipp_labels[np.where(labels.get_fdata()==72)] = 1
            hipp_labels[np.where(labels.get_fdata()==73)] = 1
            hipp_labels[np.where(labels.get_fdata()==74)] = 1
            hipp_labels[np.where(labels.get_fdata()==75)] = 1
            
            #If amygdala computed:
            #Right talamus
            #amy_labels[np.where(labels.get_fdata()==3)] = 1

            #Left talamus
            #amy_labels[np.where(labels.get_fdata()==51)] = 1
            
            #If thalamus computed:
            #Right talamus
            #tal_labels[np.where(labels.get_fdata()==43)] = 1
            #tal_labels[np.where(labels.get_fdata()==44)] = 1
            #tal_labels[np.where(labels.get_fdata()==45)] = 1

            #Left talamus
            #tal_labels[np.where(labels.get_fdata()==91)] = 1
            #tal_labels[np.where(labels.get_fdata()==92)] = 1
            #tal_labels[np.where(labels.get_fdata()==93)] = 1

            new_hipp = nib.Nifti1Image(hipp_labels, affine=labels.affine)
            new_hipp.get_data_dtype() == np.dtype(np.float64)
        
            path_3 = str(case) + "/" + str(temps)
            mask_path = os.path.join(path_,path_3)
            nib.save(new_hipp, os.path.join(mask_path, 'hipp')) 

#Check that the headers are the same and therefore there is no mismatch when working with the original image and the mask.
#hdr_img = img.header
#hdr_mask = mask.header
#print(hdr_img)
#print(" ")
#print(hdr_mask)

In [4]:
# Use a parameter file, this customizes the extraction settings and also specifies the input image types to use and which features should be extracted

params = os.path.join('..', 'examples', 'exampleSettings', 'TFG parameters.yaml')

extractor = featureextractor.RadiomicsFeatureExtractor(params)

In [5]:
for i in range (len(nom_carpetes)): 
    case = nom_carpetes[i]
    
    for temps in nom_temps:
        path_1 = str(case) + "/" + str(temps) 
        data_path = os.path.join(path_,path_1)
        
        if os.path.isdir(data_path):
            data_path_img = data_path + "/" + "t1.nii"
            img = sitk.ReadImage(data_path_img)
            
            data_path_hipp = data_path + "/" + "hipp.nii"
            mask = sitk.ReadImage(data_path_hipp)
            
            featureVector = extractor.execute(img, mask)
            #print(" ")
            
            values_firstorder = [] #myData
            values_shape = []
            values_glcm = []
            values_glrlm = []
            values_glszm = []
            values_gldm = []
            values_ngtdm = []
            features_firstorder = [] #myFeatures
            features_shape = []
            features_glcm = []
            features_glrlm = []
            features_glszm = []
            features_gldm = []
            features_ngtdm = []
            counter_firstorder = 0
            counter_shape = 0
            counter_glcm = 0
            counter_glrlm = 0
            counter_glszm = 0
            counter_gldm = 0
            counter_ngtdm = 0
            
            # Show output
            for featureName in featureVector.keys():
                #print('Computed %s: %s' % (featureName, featureVector[featureName]))
                
           #Firstorder 
                if '_firstorder_' in featureName:
                    features_firstorder.append(featureName)
                    values_firstorder.append(featureVector[featureName])
                    counter_firstorder = counter_firstorder + 1
                
                llista_firstorder = np.asarray([features_firstorder,values_firstorder])
                
                data_path_csv = data_path + "/" + "firstordertot.csv"
                firstorder = open(data_path_csv, 'w') #firstorder 
                
                with firstorder:
                    writer = csv.writer(firstorder)
                    writer.writerows(llista_firstorder)
                    
            #Shape 
                if '_shape_' in featureName:
                    features_shape.append(featureName)
                    values_shape.append(featureVector[featureName])
                    counter_shape = counter_shape + 1
                
                llista_shape = np.asarray([features_shape,values_shape])
                
                data_path_csv = data_path + "/" + "shapetot.csv"
                shape = open(data_path_csv, 'w') #shape
                
                with shape:
                    writer = csv.writer(shape)
                    writer.writerows(llista_shape)
                    
            #Glcm 
                if '_glcm_' in featureName:
                    features_glcm.append(featureName)
                    values_glcm.append(featureVector[featureName])
                    counter_glcm = counter_glcm + 1
                
                llista_glcm = np.asarray([features_glcm,values_glcm])
                
                data_path_csv = data_path + "/" + "glcmtot.csv"
                glcm = open(data_path_csv, 'w') #glcm
                
                with glcm:
                    writer = csv.writer(glcm)
                    writer.writerows(llista_glcm)
                    
            #Glrlm 
                if '_glrlm_' in featureName:
                    features_glrlm.append(featureName)
                    values_glrlm.append(featureVector[featureName])
                    counter_glrlm = counter_glrlm + 1
                
                llista_glrlm = np.asarray([features_glrlm,values_glrlm])
                
                data_path_csv = data_path + "/" + "glrlmtot.csv"
                glrlm = open(data_path_csv, 'w') #glrlm
                
                with glrlm:
                    writer = csv.writer(glrlm)
                    writer.writerows(llista_glrlm)
                    
            #Glszm 
                if '_glszm_' in featureName:
                    features_glszm.append(featureName)
                    values_glszm.append(featureVector[featureName])
                    counter_glszm = counter_glszm + 1
                
                llista_glszm = np.asarray([features_glszm,values_glszm])
                
                data_path_csv = data_path + "/" + "glszmtot.csv"
                glszm = open(data_path_csv, 'w') #glszm
                
                with glszm:
                    writer = csv.writer(glszm)
                    writer.writerows(llista_glszm)
                    
            #Gldm 
                if '_gldm_' in featureName:
                    features_gldm.append(featureName)
                    values_gldm.append(featureVector[featureName])
                    counter_gldm = counter_gldm + 1
                
                llista_gldm = np.asarray([features_gldm,values_gldm])
                
                data_path_csv = data_path + "/" + "gldmtot.csv"
                gldm = open(data_path_csv, 'w') #gldm
                
                with gldm:
                    writer = csv.writer(gldm)
                    writer.writerows(llista_gldm)
                    
            #Ngtdm 
                if '_ngtdm_' in featureName:
                    features_ngtdm.append(featureName)
                    values_ngtdm.append(featureVector[featureName])
                    counter_ngtdm = counter_ngtdm + 1
                
                llista_ngtdm = np.asarray([features_ngtdm,values_ngtdm])
                
                data_path_csv = data_path + "/" + "ngtdmtot.csv"
                ngtdm = open(data_path_csv, 'w') #ngtdm
                
                with ngtdm:
                    writer = csv.writer(ngtdm)
                    writer.writerows(llista_ngtdm)
                    
print(counter_firstorder)
print(counter_shape)
print(counter_glcm)
print(counter_glrlm)
print(counter_glszm)
print(counter_gldm)
print(counter_ngtdm)

GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Avera

GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Avera

GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Avera

18
14
24
16
16
14
5
